In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h1>
<font color="#ff2e2e">
Mask Overlay
</font>
</h1>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import dlib
from google.colab.patches import cv2_imshow

<h3><font color = "#079e05"> This function is used to plot your result </font></h3>

In [ ]:
def plotter(img_list, r, w, gray, wr, hr, fig_name = None):
    '''
    Plots images' list with its' caption and saves result image if you want.

    Parameters:
        img_list (list): The list of tuples of image and its' caption.
        r (int): The number of row(s).
        w (int): The number of colunm(s).
        gray (bool): The flag for plotting images in grayscale mode.
        wr (int): The width of one figure.
        hr (int): The height of one figure.
        fig_name (str): The name of the image of the plot. if not set this parameter the plot doesn't save.
    '''
    
    plt.rcParams['figure.figsize'] = (wr, hr)
    for i in range(len(img_list)):
        plt.subplot(r, w, i + 1)
        if img_list[i][2] == 'img':
            if gray:
                plt.imshow(img_list[i][0], cmap = 'gray')
            else:
                plt.imshow(img_list[i][0])
            plt.xticks([])
            plt.yticks([])
        elif img_list[i][2] == 'hist':
            plt.bar(np.arange(len(img_list[i][0])), img_list[i][0], color = 'c')
        else:
            raise Exception("Only image or histogram. Use third parameter of tuples in img_list and set it to img or hist.")
        plt.title(img_list[i][1])
    if fig_name is not None:
        plt.savefig(fig_name + '.png')
    plt.show()

<h3><font color = "#079e05"> Complete put_mask function </font></h3>

In [ ]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK

!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2

datFile =  "/content/shape_predictor_68_face_landmarks.dat"

--2023-04-29 18:26:41--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  16.5MB/s    in 3.7s    

2023-04-29 18:26:45 (16.5 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
def put_mask(face, mask):
    '''
    Adds mask image on face image.
    
    Parameters:
        face (numpy.ndarray): face image.
        mask (numpy.ndarray): mask image.
    
    Returns:
        numpy.ndarray: The result image.
    '''
    #############
    # Your code #
    #############
    result = face.copy()
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(datFile)
    rects = detector(face, 0)
    landmarks = predictor(face, rects[0])

    dst_points = np.array([
        (landmarks.part(1).x, landmarks.part(1).y), 
        (landmarks.part(15).x, landmarks.part(15).y),
        (landmarks.part(28).x, landmarks.part(28).y),
        (landmarks.part(8).x, landmarks.part(8).y)
    ], dtype=np.float32)

    src_points = np.array([
        (80, 220),
        (1020, 221),
        (615, 105),
        (615, 700),
    ], dtype=np.float32)
    
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    mask_warped = cv2.warpPerspective(mask, M, (result.shape[1], result.shape[0]))

    mask_gray = cv2.cvtColor(mask_warped, cv2.COLOR_BGR2GRAY)
    _, mask_binary = cv2.threshold(mask_gray, 10, 255, cv2.THRESH_BINARY)
    mask_binary_inv = cv2.bitwise_not(mask_binary)
    face_masked = cv2.bitwise_and(face, face, mask=mask_binary_inv)
    mask_warped_masked = cv2.bitwise_and(mask_warped, mask_warped, mask=mask_binary)
    result = cv2.add(face_masked, mask_warped_masked)

    cv2_imshow(mask_warped)
    
    return result

<h3><font color = "#079e05"> Show the result </font></h3>

In [ ]:
image_list = []
path = '/content/drive/MyDrive/ColabNotebooks/FCV/HW3/images/Q3/'
face = cv2.cvtColor(cv2.imread(os.path.join(path, 'face.jpg')), cv2.COLOR_BGR2RGB)
mask = cv2.cvtColor(cv2.imread(os.path.join(path, 'mask.jpg')), cv2.COLOR_BGR2RGB)
image_list.append([face, 'face', 'img'])
image_list.append([put_mask(face, mask), 'result', 'img'])
plotter(image_list, 1, 3, True, 20, 10)